In [1]:
from azureml.core import Workspace, Experiment
import pandas as pd



ws = Workspace.from_config()

# ws = Workspace.get(name='quick-starts-ws-131305')

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-131305
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-131305


In [2]:
# !pip install --upgrade --force-reinstall -r https://aka.ms/automl_linux_requirements.txt

In [3]:
# !pip install --upgrade --force-reinstall -r https://automlcesdkdataresources.blob.core.windows.net/validated-requirements/1.19.0/validated_linux_requirements.txt

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster

cpu_cluster_name = 'cpucluster'

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


### YOUR CODE HERE ###

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C" : uniform(0.1,1),
                            "--max_iter": choice(50,100,150,200)})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1,evaluation_interval = 1,delay_evaluation=5)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./training",entry_script="train.py",compute_target=cpu_cluster_name)
### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =  HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    max_total_runs=100,
    max_concurrent_runs=4,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=policy,
    estimator=est
)

### YOUR CODE HERE ###

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

experiment=Experiment(ws,ws.name)
hyperdrive_run = experiment.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)
assert(hyperdrive_run.get_status() == "Completed")
### YOUR CODE HERE ###

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_cc90ab6c-9ed0-4b88-86dc-22da51543144
Web View: https://ml.azure.com/experiments/quick-starts-ws-131305/runs/HD_cc90ab6c-9ed0-4b88-86dc-22da51543144?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-131305/workspaces/quick-starts-ws-131305

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-21T21:33:10.012063][API][INFO]Experiment created<END>\n""<START>[2020-12-21T21:33:10.554297][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-12-21T21:33:10.731061][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-21T21:33:12.4547415Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_cc90ab6c-9ed0-4b88-86dc-22da51543144
Web View: https://ml.azure.com/experiments/quick-starts-ws-131305/runs/HD_cc90ab6c-9ed0-4b88-86dc-22da51543144?wsid=/subsc

In [7]:
import joblib
import os
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength: ',parameter_values[1])
print('\n Max iterations: ',parameter_values[3])


### YOUR CODE HERE ###

Best Run Id:  HD_cc90ab6c-9ed0-4b88-86dc-22da51543144_90

 Accuracy: 0.9103692463328276

 Regularization Strength:  0.9072469401405283

 Max iterations:  150


In [8]:
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_387193fc23067ec1ff25f7e6d85fccc4a06761900e9333365117cb3d2f259a58_d.txt',
 'azureml-logs/65_job_prep-tvmps_387193fc23067ec1ff25f7e6d85fccc4a06761900e9333365117cb3d2f259a58_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_387193fc23067ec1ff25f7e6d85fccc4a06761900e9333365117cb3d2f259a58_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/104_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/engine_spans_l_06f140cb-22a1-4cfe-9900-38a10b364172.jsonl',
 'logs/azureml/dataprep/python_span_l_06f140cb-22a1-4cfe-9900-38a10b364172.jsonl',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/model.joblib']

In [9]:
best_model = best_run.register_model(model_name = 'hyperdrive_run_bankmkt', model_path = 'outputs/model.joblib')

best_run.download_file("/outputs/model.joblib", "./outputs/hyperdrive_run_bankmkt.joblib")

In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
from azureml.core import Workspace, Dataset

# dataset = Dataset.get_by_name(ws, name='Bank-marketing')
# ds = dataset.to_pandas_dataframe()


data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(path=data_path)
ds = dataset.to_pandas_dataframe()

### YOUR CODE HERE ###

In [11]:
from training.train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

df = x.join(y)


train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
label = "y"

In [12]:
from azureml.core import Dataset
#Export df to csv

path = './outputs/'

train_data.to_csv(path+'train_data.csv')
test_data.to_csv(path+'test_data.csv')

#Import csv as dataset
datastore=ws.get_default_datastore()
datastore.upload(src_dir=path)

training_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('train_data.csv'))])
test_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('test_data.csv'))])

Uploading an estimated of 3 files
Uploading ./outputs/hyperdrive_run_bankmkt.joblib
Uploaded ./outputs/hyperdrive_run_bankmkt.joblib, 1 files out of an estimated total of 3
Uploading ./outputs/test_data.csv
Uploaded ./outputs/test_data.csv, 2 files out of an estimated total of 3
Uploading ./outputs/train_data.csv
Uploaded ./outputs/train_data.csv, 3 files out of an estimated total of 3
Uploaded 3 files


In [13]:
df.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=training_dataset,
    label_column_name=label,
    n_cross_validations=5,
    compute_target = cpu_cluster)

In [15]:
# Submit your automl run
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails
# experiment=Experiment(ws,ws.name)

exp = Experiment(workspace=ws, name="AutoML-project")
automl_run = exp.submit(config = automl_config)


RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output = True)
assert(automl_run.get_status() == "Completed")


### YOUR CODE HERE ###

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.


In [20]:
# Retrieve and save your best automl model.
automl_best_run, automl_fitted_model = automl_run.get_output()


# Reference: https://knowledge.udacity.com/questions/357775 
print("Fitted model: ", automl_fitted_model.steps[-1])
print("---"*5)
print("AutoML best run accuracy: ", automl_best_run.get_metrics(name="accuracy"))
print("---"*5)
print("AutoML run Summary: ", automl_run.summary())


### YOUR CODE HERE ###

Fitted model:  ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('0',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=1.0,
                                                                               importance_type='split',
                                                                               learning_rate=0.1,
                                  

In [21]:

automl_best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')
joblib.dump(automl_fitted_model, "./outputs/automl_fitted_model.joblib")

['./outputs/automl_fitted_model.joblib']

In [23]:
automl_estimator = automl_fitted_model.steps[-1]
with open("./project.txt", 'w') as output:
    output.write(str(automl_estimator))

In [24]:
cpu_cluster.delete()

ComputeTargetException: ComputeTargetException:
	Message: Received bad response from Resource Provider:
Response Code: 500
Headers: {'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Length': '1456', 'Content-Type': 'application/json; charset=utf-8', 'Expires': '-1', 'x-ms-failure-cause': 'service', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-response-type': 'error', 'x-ms-client-request-id': '3402c6ae-69ee-483d-9744-abc5dad58af6', 'x-ms-client-session-id': 'cc5104e1-2a6d-4d42-8fd6-cf358382c5aa', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.274', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'x-ms-ratelimit-remaining-subscription-deletes': '14999', 'x-ms-request-id': 'eff90bcc-2fa6-439e-b08d-0676347dba4a', 'x-ms-correlation-request-id': 'eff90bcc-2fa6-439e-b08d-0676347dba4a', 'x-ms-routing-request-id': 'SOUTHCENTRALUS:20201221T231133Z:eff90bcc-2fa6-439e-b08d-0676347dba4a', 'Date': 'Mon, 21 Dec 2020 23:11:33 